In [ ]:
#%%

import os
os.environ["QT_API"] = "pyqt"

import numpy as np
import mayavi.mlab as mlab
import scipy
import skimage
import skimage.transform
from skimage.io import imread
from scipy.ndimage.filters import convolve
#import matplotlib.pyplot as plt

from tqdm import tqdm

#%%

def readTerrain(name,pixelsize=0.02):
    """Returns a tuple terrain,mask with the requested pixelsize
    Values in terrain (2D np float array) are interpreted as meters
    Values in mask (2D np float array) are interpreted as probabilities [0-1]
    """
    
    if(name=="quarry"):
        tpixelsize=0.02
        im=imread("/home/lale/Downloads/quarry_cropped4_scaled2cm.png")/2**16*20
        noise=0.0 # 0: no noise
        im=im+np.random.rand(*im.shape)*noise
        sz=1.5 # 0: no filtering
        im=scipy.ndimage.filters.gaussian_filter(im, sz, truncate=2) # careful, will create problems with nans
        
    elif(name=="uzh"):
        im=imread("/home/lale/Downloads/uzh_elevation_1.png")[:,:,0].astype(float)/255*5
        im[im==0]=np.nan
        im=skimage.transform.resize(im,np.array(im.shape)*5)
        tpixelsize=0.05
        im=scipy.ndimage.filters.gaussian_filter(im, 3, truncate=1)
        
    elif(name=="custom1"):
        im=imread("./custom1.png")[:,:,0].astype(float)/255*2
        tpixelsize=0.02
        
    else:
        assert(False)
    
    im=skimage.transform.resize(im,(np.array(im.shape)*tpixelsize/pixelsize).astype(int))
    im=im-np.nanmin(im)
    
    return im


def makeRobot(x,y,z,length):
    vertices=np.array([[0,0,1],[1,0,0],[0,3,0],[-1,0,0]]).astype(float)/3*length+np.array([x,y,z])
    triangles=[[0,1,3],[0,1,2],[0,3,2],[3,1,2]]
    mesh=mlab.triangular_mesh(vertices[:,0], vertices[:,1], vertices[:,2], triangles, color=(0.3,0.3,0.6))
    mesh.actor.property.interpolation = 'phong'
    mesh.actor.property.specular = 0.2
    mesh.actor.property.specular_power = 10
    mesh.actor.property.ambient_color = (1,1,1)
    mesh.actor.property.diffuse_color = (0.3,0.3,0.6)
    mesh.actor.property.color = (0.3,0.3,0.6)
    mesh.actor.property.ambient = 0.1
    return()

def visualizeTerrain(terrain, pixelsize, mask=None, fig=None):
    """Original version: not very flexible to display traversability mask.
    Use visualizeTerrain2 instead if you have a mask"""
    
    if(fig is None):
        fig = mlab.figure()
    
    fig.scene.background = (1,1,1)
    
    y, x = np.meshgrid(np.arange(terrain.shape[0])*pixelsize,np.arange(terrain.shape[1])*pixelsize)
    s = mlab.surf(x, y, terrain, color=(1.0,1.0,1.0))
    
    s.actor.property.interpolation = 'phong'
    s.actor.property.specular = 0.0
    s.actor.property.specular_power = 10
    s.actor.property.ambient_color = (1,1,1)
    s.actor.property.diffuse_color = (0.7,0.7,0.7)
    s.actor.property.color = (0.7,0.7,0.7)
    s.actor.property.ambient = 0.02
    
    if(mask is not None):
        terrain_mask=terrain.copy()
        terrain_mask[mask<0.9]=np.nan
        sm = mlab.surf(x, y, terrain_mask+0.010, color=(1.0,1.0,1.0))
    
        sm.actor.property.interpolation = 'phong'
        sm.actor.property.specular = 0.2
        sm.actor.property.specular_power = 10
        sm.actor.property.ambient_color = (0.6,1,0.6)
        sm.actor.property.diffuse_color = (0.6,0.9,0.6)
        sm.actor.property.color = (0.6,0.9,0.6)
        sm.actor.property.opacity = 0.7
        sm.actor.property.ambient = 0.05
        
        terrain_mask=terrain.copy()
        terrain_mask[mask<0.2]=np.nan
        sm = mlab.surf(x, y, terrain_mask+0.005, color=(1.0,1.0,1.0))
    
        sm.actor.property.interpolation = 'phong'
        sm.actor.property.specular = 0.2
        sm.actor.property.specular_power = 10
        sm.actor.property.ambient_color = (0.9,0.9,0.6)
        sm.actor.property.diffuse_color = (0.9,0.9,0.6)
        sm.actor.property.color = (0.8,0.8,0.6)
        sm.actor.property.opacity = 0.5
        sm.actor.property.ambient = 0.05
    
    square=np.array([[0,0],[1,0],[1,1],[0,1]])[[0,1,2,3,0],:]
    square=np.hstack((square*np.array([[np.max(x),np.max(y)]]), np.zeros((5,1))))
    base=mlab.plot3d(square[:,0], square[:,1], square[:,2], color=(0,0,0), line_width=2)
    for i in range(4):
        p=np.mean(square[[i,i+1],:],axis=0)
        d=np.linalg.norm(square[i+1,:]-square[i+0,:])
        mlab.text3d(p[0], p[1], p[2], "{:.1f}m".format(d), scale=0.4, color=(0,0,0))
    height=mlab.plot3d(np.array([0.0,0.0]), np.array([0.0,0.0]), np.array([0.0,np.nanmax(terrain)]), color=(0,0,0), line_width=2)
    mlab.text3d(0.0, 0.0, np.nanmax(terrain)/2, "{:.1f}m".format(np.nanmax(terrain)), scale=0.4, color=(0,0,0))
    
    makeRobot(-1,0,0,0.8)
    mlab.text3d(-1, -1, 0, "robot", scale=0.4, color=(0,0,0))
    
    if(False):
        # function returning height of an xy point (in meters) sampled on im
        heightof=scipy.interpolate.RectBivariateSpline(x[:, 0], y[0, :], im)
        
        # coordinates in meters on map of my waypoints
        waypoints = np.array([[1,1],[30,5],[5,30]])
        
        ls=np.cumsum([0]+[np.linalg.norm(end-start) for start,end in zip(waypoints[0:-1],waypoints[1:])])
        psxy=scipy.interpolate.interp1d(ls,waypoints,axis=0)(np.arange(0,ls[-1],
                                      0.2)) # sample one point every 20 cm
        # ps are coordinates of dense points sampled along the path defined by waypoints
        psz=heightof(psxy[:,0],psxy[:,1],grid=False)
        
        mlab.plot3d(psxy[:,0], psxy[:,1], psz + 0.5, color=(0.5,0.5,0.8), tube_radius=0.1)
    
    from tvtk.api import tvtk
    fig.scene.interactor.interactor_style = tvtk.InteractorStyleTerrain()
    #outline=mlab.outline(s,color=(0,0,0))
    #axes=mlab.axes(s,color=(0,0,0))


def visualizeTerrain2(terrain, pixelsize, mask, fig=None):
    """Draws a surface colored according to mask using a custom colormap
    Inspired by http://gael-varoquaux.info/programming/mayavi-representing-an-additional-scalar-on-surfaces.html """
    
    if(fig is None):
        fig = mlab.figure()
    
    fig.scene.background = (1,1,1)
    
    y, x = np.meshgrid(np.arange(terrain.shape[0])*pixelsize,np.arange(terrain.shape[1])*pixelsize)
    s = mlab.mesh(x, y, terrain, scalars=mask) # We use a mesh, not a surf. Note: it's less efficient
    
    # http://docs.enthought.com/mayavi/mayavi/auto/example_custom_colormap.html
    colormap=np.tile(np.array([180,180,180,255]),[256,1]) # all gray
    colormap[:,1]=np.linspace(180,220,256) # scale green channel
    s.module_manager.scalar_lut_manager.lut.table = colormap
    s.module_manager.scalar_lut_manager.lut.range=np.array([0.0,1.0])
    s.actor.property.interpolation = 'phong'
    s.actor.property.specular = 0.0
    s.actor.property.specular_power = 10
    s.actor.property.ambient_color = (1,1,1)

    s.actor.property.ambient = 0.02
    
    square=np.array([[0,0],[1,0],[1,1],[0,1]])[[0,1,2,3,0],:]
    square=np.hstack((square*np.array([[np.max(x),np.max(y)]]), np.zeros((5,1))))
    base=mlab.plot3d(square[:,0], square[:,1], square[:,2], color=(0,0,0), line_width=2)
    for i in range(4):
        p=np.mean(square[[i,i+1],:],axis=0)
        d=np.linalg.norm(square[i+1,:]-square[i+0,:])
        mlab.text3d(p[0], p[1], p[2], "{:.1f}m".format(d), scale=0.4, color=(0,0,0))
    height=mlab.plot3d(np.array([0.0,0.0]), np.array([0.0,0.0]), np.array([0.0,np.nanmax(terrain)]), color=(0,0,0), line_width=2)
    mlab.text3d(0.0, 0.0, np.nanmax(terrain)/2, "{:.1f}m".format(np.nanmax(terrain)), scale=0.4, color=(0,0,0))
    
    makeRobot(-1,0,0,0.8)
    mlab.text3d(-1, -1, 0, "robot", scale=0.4, color=(0,0,0))
    
    if(False):
        # function returning height of an xy point (in meters) sampled on im
        heightof=scipy.interpolate.RectBivariateSpline(x[:, 0], y[0, :], im)
        
        # coordinates in meters on map of my waypoints
        waypoints = np.array([[1,1],[30,5],[5,30]])
        
        ls=np.cumsum([0]+[np.linalg.norm(end-start) for start,end in zip(waypoints[0:-1],waypoints[1:])])
        psxy=scipy.interpolate.interp1d(ls,waypoints,axis=0)(np.arange(0,ls[-1],
                                      0.2)) # sample one point every 20 cm
        # ps are coordinates of dense points sampled along the path defined by waypoints
        psz=heightof(psxy[:,0],psxy[:,1],grid=False)
        
        mlab.plot3d(psxy[:,0], psxy[:,1], psz + 0.5, color=(0.5,0.5,0.8), tube_radius=0.1)
    
    from tvtk.api import tvtk
    fig.scene.interactor.interactor_style = tvtk.InteractorStyleTerrain()
    #outline=mlab.outline(s,color=(0,0,0))
    #axes=mlab.axes(s,color=(0,0,0))

    
def visualizePatch(patch, pixelsize, fig=None, robotlength=0.8):
    
    if(fig is None):
        fig = mlab.figure()
    
    fig.scene.background = (1,1,1)
    
    patch=patch-np.nanmin(patch)
    y, x = np.meshgrid(np.arange(patch.shape[0])*pixelsize,np.arange(patch.shape[1])*pixelsize)
    s = mlab.surf(x, y, patch, color=(1.0,1.0,1.0))
    s.actor.property.interpolation = 'phong'
    s.actor.property.specular = 0.0
    s.actor.property.specular_power = 10
    s.actor.property.ambient_color = (1,1,1)
    s.actor.property.diffuse_color = (0.7,0.7,0.7)
    s.actor.property.color = (0.7,0.7,0.7)
    s.actor.property.ambient = 0.02
    
    #sw = mlab.surf(x, y, patch+0.001, color=(0,0,0), line_width=1, opacity=0.1)
    #sw.actor.property.representation = 'wireframe'
    
    square=np.array([[0,0],[1,0],[1,1],[0,1]])[[0,1,2,3,0],:]
    square=np.hstack((square*np.array([[np.max(x),np.max(y)]]), np.zeros((5,1))))
    base=mlab.plot3d(square[:,0], square[:,1], square[:,2], color=(0,0,0), line_width=2, tube_radius=None)
    for i in range(4):
        p=np.mean(square[[i,i+1],:],axis=0)
        d=np.linalg.norm(square[i+1,:]-square[i+0,:])
        #mlab.text3d(p[0], p[1], p[2], "{:.1f}m".format(d), scale=0.4, color=(0,0,0))
    #height=mlab.plot3d(np.array([0.0,0.0]), np.array([0.0,0.0]), np.array([0.0,np.nanmax(patch)]), color=(0,0,0), line_width=2)
    #mlab.text3d(0.0, 0.0, np.nanmax(patch)/2, "{:.1f}m".format(np.nanmax(patch)), scale=0.4, color=(0,0,0))
    
    for xi in [int(np.floor(k)) for k in np.linspace(0,x.shape[1]-1,5)]:
        mlab.plot3d(x[:,xi],y[:,xi],patch[:,xi]+0.002, color=(0,0,0), line_width=1, tube_radius=None)
    for yi in [int(np.floor(k)) for k in np.linspace(0,x.shape[0]-1,5)]:
        mlab.plot3d(x[yi,:],y[yi,:],patch[yi,:]+0.002, color=(0,0,0), line_width=1, tube_radius=None)
    
    makeRobot(np.mean(x),-robotlength*1.2,0,length=robotlength)
    #mlab.text3d(-1, -1, 0, "robot", scale=0.4, color=(0,0,0))
    
    mlab.view(azimuth=-90, elevation=45)
    
    from tvtk.api import tvtk
    fig.scene.interactor.interactor_style = tvtk.InteractorStyleTerrain()
    #outline=mlab.outline(s,color=(0,0,0))
    #axes=mlab.axes(s,color=(0,0,0))


def saveAnimation(fig, directory, motion="360"):
    """Saves an animation from the passed mlib figure in the provided directory
    Will create directory of it does not exist
    Will remove any previous frames in the directory if any exists
    Individual frames saved as PNG
    motion in ["360", "oscillate"]
    Motion begins at initial view in figure
    """
    
    import pathlib
    directory=pathlib.Path(directory) # Converts in case it's a string
    directory.mkdir(exist_ok=True)
    assert(directory.is_dir())
    [f.unlink() for f in directory.glob("anim*.png")] # remove any preexisting animation in the directory
    
    (original_azimuth, elevation, distance, focalpoint)=mlab.view(figure=fig)
    
    azimuths={"360":        np.arange(0,360,2)+original_azimuth,
              "oscillate":  np.sin(np.linspace(0,2*np.pi,100,endpoint=False))*30+original_azimuth}[motion]
    for i,a in tqdm(list(enumerate(azimuths))):
        mlab.view(azimuth=a,figure=fig)
        mlab.savefig(str(directory/'anim{:04d}.png'.format(i)))
    
    import subprocess
    subprocess.call(["convert", "-delay", "1x20", "-loop", "0", str(directory)+"/anim*.png", str(directory)+"/anim.gif"])

#%%
pixelsize=0.02
terrain=readTerrain("quarry",pixelsize)

#%%
mlab.close(all=True)
mlab.options.offscreen = True # To avoid opening a window
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(terrain,pixelsize,fig=fig)
saveAnimation(fig, "terrain-360", motion="360")
saveAnimation(fig, "terrain-osc", motion="oscillate")
mlab.close(fig)

#%%
mlab.close(all=True)
mlab.options.offscreen = True # To avoid opening a window
fig=mlab.figure(size=(400,400))
visualizePatch(terrain[100:170,100:170],pixelsize,fig=fig)
saveAnimation(fig, "testpatch-360", motion="360")
saveAnimation(fig, "testpatch-osc", motion="oscillate")
mlab.close(fig)
#%%

terrain=readTerrain(name="quarry")
mask=imread("/home/lale/Downloads/min_traversability_map_color_quarry.png")[:,:,3].astype(float)/255
mlab.options.offscreen = False # To avoid opening a window
fig=mlab.figure(size=(1000,1000))
#visualizeTerrain(terrain,pixelsize,mask=mask,fig=fig)
visualizeTerrain2(terrain,pixelsize,mask=mask,fig=fig)


#%%
terrain=readTerrain(name="custom1")
mlab.options.offscreen = False # To avoid opening a window
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(terrain,pixelsize,fig=fig)

#%%
im=imread("disp4.jpg").astype(float)/255*0.6
im=skimage.transform.resize(im,(1024,1024))
im=scipy.ndimage.filters.gaussian_filter(im, 2, truncate=2)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,0.02,fig=fig)
#%%
im=imread("disp3.jpg")[:,:,0].astype(float)/255*0.1
im=skimage.transform.resize(im,(1024,1024))
im=scipy.ndimage.filters.gaussian_filter(im, 1.3, truncate=2)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,0.005,fig=fig)
#%%
im=imread("photosculpt00763-displacement.jpg").astype(float)/255*0.3
im=skimage.transform.resize(im,(1024,1024))
im=scipy.ndimage.filters.gaussian_filter(im, 1.3, truncate=2)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,0.005,fig=fig)

#%% Realistic forest floor (useless for simulation)
import skimage
im=imread("textures/rd-textures.com/MUDDY-FOREST-FLOOR/MUDDY-FOREST-FLOOR-01_DEPTH_8k_3x3x0.06.tif")
im=skimage.img_as_float(im)*0.06 # Depth of full image range
pixelsize=0.002 # we rescale to 2mm per pixel (makes no sense for robot, in fact)
texturearea=np.array([3.0,3.0]) # texture represents 3m x 3m
im=skimage.transform.resize(im,texturearea/pixelsize)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,pixelsize,fig=fig)

#%% asphalt tiled 4x4
import skimage
im=imread("textures/rd-textures.com/ASPHALT/ASPHALT-02_Depth_8k_2.5x2.5x0.05.tif")
im=skimage.img_as_float(im)*0.05 # Depth of full image range
pixelsize=0.02 # we rescale to 2cm per pixel
texturearea=np.array([2.5,2.5]) # texture represents 2.5m x 2.5m
repeats=[4,4]
im=np.tile(skimage.transform.resize(im,texturearea/pixelsize),repeats)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,pixelsize,fig=fig)

#%% rock (as dimensioned)
im=imread("textures/rd-textures.com/ROCK-02/ROCK-02_200cm_Depth_8k.tif")
im=skimage.img_as_float(im)*0.1 # Depth of full image range (unknown)
pixelsize=0.02 # we rescale to 2cm per pixel
texturearea=np.array([2.0,2.0]) # texture represents 2.0m x 2.0m
repeats=[2,2]
im=np.tile(skimage.transform.resize(im,texturearea/pixelsize),repeats)
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,pixelsize,fig=fig)

#%% rock (rescaled in order to be useful)
im=imread("textures/rd-textures.com/ROCK-02/ROCK-02_200cm_Depth_8k.tif")
im=skimage.img_as_float(im)*0.5 # Depth of full image range (unknown)
pixelsize=0.02 # we rescale to 2cm per pixel
texturearea=np.array([10.0,10.0]) # texture represents 2.0m x 2.0m, but we make it much larger
repeats=[2,2]
im=np.tile(skimage.transform.resize(im,texturearea/pixelsize),repeats)
im=im*np.linspace(0,1,im.shape[0])
im=im+np.linspace(0,1,im.shape[0])[:,np.newaxis]**2*8
fig=mlab.figure(size=(1000,1000))
visualizeTerrain(im,pixelsize,fig=fig)


In [2]:
#%%

import os
os.environ["QT_API"] = "pyqt"

import numpy as np
import mayavi.mlab as mlab
import scipy
import skimage
import skimage.transform
from skimage.io import imread
from scipy.ndimage.filters import convolve
#import matplotlib.pyplot as plt

from tqdm import tqdm

def visualizePatch(patch, pixelsize, fig=None, robotlength=0.8):
    
    if(fig is None):
        fig = mlab.figure()
    
    fig.scene.background = (1,1,1)
    
    patch=patch-np.nanmin(patch)
    y, x = np.meshgrid(np.arange(patch.shape[0])*pixelsize,np.arange(patch.shape[1])*pixelsize)
    s = mlab.surf(x, y, patch, color=(1.0,1.0,1.0))
    s.actor.property.interpolation = 'phong'
    s.actor.property.specular = 0.0
    s.actor.property.specular_power = 10
    s.actor.property.ambient_color = (1,1,1)
    s.actor.property.diffuse_color = (0.7,0.7,0.7)
    s.actor.property.color = (0.7,0.7,0.7)
    s.actor.property.ambient = 0.02
    
    #sw = mlab.surf(x, y, patch+0.001, color=(0,0,0), line_width=1, opacity=0.1)
    #sw.actor.property.representation = 'wireframe'
    
    square=np.array([[0,0],[1,0],[1,1],[0,1]])[[0,1,2,3,0],:]
    square=np.hstack((square*np.array([[np.max(x),np.max(y)]]), np.zeros((5,1))))
    base=mlab.plot3d(square[:,0], square[:,1], square[:,2], color=(0,0,0), line_width=2, tube_radius=None)
    for i in range(4):
        p=np.mean(square[[i,i+1],:],axis=0)
        d=np.linalg.norm(square[i+1,:]-square[i+0,:])
        #mlab.text3d(p[0], p[1], p[2], "{:.1f}m".format(d), scale=0.4, color=(0,0,0))
    #height=mlab.plot3d(np.array([0.0,0.0]), np.array([0.0,0.0]), np.array([0.0,np.nanmax(patch)]), color=(0,0,0), line_width=2)
    #mlab.text3d(0.0, 0.0, np.nanmax(patch)/2, "{:.1f}m".format(np.nanmax(patch)), scale=0.4, color=(0,0,0))
    
    for xi in [int(np.floor(k)) for k in np.linspace(0,x.shape[1]-1,5)]:
        mlab.plot3d(x[:,xi],y[:,xi],patch[:,xi]+0.002, color=(0,0,0), line_width=1, tube_radius=None)
    for yi in [int(np.floor(k)) for k in np.linspace(0,x.shape[0]-1,5)]:
        mlab.plot3d(x[yi,:],y[yi,:],patch[yi,:]+0.002, color=(0,0,0), line_width=1, tube_radius=None)
        #mlab.text3d(-1, -1, 0, "robot", scale=0.4, color=(0,0,0))
    
    mlab.view(azimuth=-90, elevation=45)
    
def visualizeTerrain(terrain, pixelsize, mask=None, fig=None):
    """Original version: not very flexible to display traversability mask.
    Use visualizeTerrain2 instead if you have a mask"""
    
    if(fig is None):
        fig = mlab.figure()
    
    fig.scene.background = (1,1,1)
    
    y, x = np.meshgrid(np.arange(terrain.shape[0])*pixelsize,np.arange(terrain.shape[1])*pixelsize)
    s = mlab.surf(x, y, terrain, color=(1.0,1.0,1.0))
    
    s.actor.property.interpolation = 'phong'
    s.actor.property.specular = 0.0
    s.actor.property.specular_power = 10
    s.actor.property.ambient_color = (1,1,1)
    s.actor.property.diffuse_color = (0.7,0.7,0.7)
    s.actor.property.color = (0.7,0.7,0.7)
    s.actor.property.ambient = 0.02
    
    if(mask is not None):
        terrain_mask=terrain.copy()
        terrain_mask[mask<0.9]=np.nan
        sm = mlab.surf(x, y, terrain_mask+0.010, color=(1.0,1.0,1.0))
    
        sm.actor.property.interpolation = 'phong'
        sm.actor.property.specular = 0.2
        sm.actor.property.specular_power = 10
        sm.actor.property.ambient_color = (0.6,1,0.6)
        sm.actor.property.diffuse_color = (0.6,0.9,0.6)
        sm.actor.property.color = (0.6,0.9,0.6)
        sm.actor.property.opacity = 0.7
        sm.actor.property.ambient = 0.05
        
        terrain_mask=terrain.copy()
        terrain_mask[mask<0.2]=np.nan
        sm = mlab.surf(x, y, terrain_mask+0.005, color=(1.0,1.0,1.0))
    
        sm.actor.property.interpolation = 'phong'
        sm.actor.property.specular = 0.2
        sm.actor.property.specular_power = 10
        sm.actor.property.ambient_color = (0.9,0.9,0.6)
        sm.actor.property.diffuse_color = (0.9,0.9,0.6)
        sm.actor.property.color = (0.8,0.8,0.6)
        sm.actor.property.opacity = 0.5
        sm.actor.property.ambient = 0.05
    
    square=np.array([[0,0],[1,0],[1,1],[0,1]])[[0,1,2,3,0],:]
    square=np.hstack((square*np.array([[np.max(x),np.max(y)]]), np.zeros((5,1))))
    base=mlab.plot3d(square[:,0], square[:,1], square[:,2], color=(0,0,0), line_width=2)
    for i in range(4):
        p=np.mean(square[[i,i+1],:],axis=0)
        d=np.linalg.norm(square[i+1,:]-square[i+0,:])
        mlab.text3d(p[0], p[1], p[2], "{:.1f}m".format(d), scale=0.4, color=(0,0,0))
    height=mlab.plot3d(np.array([0.0,0.0]), np.array([0.0,0.0]), np.array([0.0,np.nanmax(terrain)]), color=(0,0,0), line_width=2)
    mlab.text3d(0.0, 0.0, np.nanmax(terrain)/2, "{:.1f}m".format(np.nanmax(terrain)), scale=0.4, color=(0,0,0))
    
#     makeRobot(-1,0,0,0.8)
#     mlab.text3d(-1, -1, 0, "robot", scale=0.4, color=(0,0,0))
    
    if(False):
        # function returning height of an xy point (in meters) sampled on im
        heightof=scipy.interpolate.RectBivariateSpline(x[:, 0], y[0, :], im)
        
        # coordinates in meters on map of my waypoints
        waypoints = np.array([[1,1],[30,5],[5,30]])
        
        ls=np.cumsum([0]+[np.linalg.norm(end-start) for start,end in zip(waypoints[0:-1],waypoints[1:])])
        psxy=scipy.interpolate.interp1d(ls,waypoints,axis=0)(np.arange(0,ls[-1],
                                      0.2)) # sample one point every 20 cm
        # ps are coordinates of dense points sampled along the path defined by waypoints
        psz=heightof(psxy[:,0],psxy[:,1],grid=False)
        
        mlab.plot3d(psxy[:,0], psxy[:,1], psz + 0.5, color=(0.5,0.5,0.8), tube_radius=0.1)
    
    from tvtk.api import tvtk
    fig.scene.interactor.interactor_style = tvtk.InteractorStyleTerrain()

In [ ]:
from utilities.postprocessing.handlers.functional import read_image

hm = read_image('/home/francesco/Documents/Master-Thesis/core/maps/new-train/bumps0-rocks0.png')

visualizeTerrain(hm, 0.02)